# Build Your first Agent

## Use Case

  - Build a spring boot document reader and online article suggestion agent based on RAG Agentic implementation.

### Goal of the Agent

  - Agent will accept the user query identify if the question is Spring Boot Documentation specific or not.

  - If the query is Technical in nature and Spring Boot specific query then it will get the query specific context information from Chroma DB spcecific DB and make LLM call to summarize the user query.

  - If the query is not technical in nature and not specific to Spring Boot then answer I don't know.

  - This agent will be extend to do a wen search and identify user question specific articles from specific websites only. This use case is yet to be implemented.



## Agent Architecture overview


### Document Loader Componenet

   - As this use case demands RAG based Agent. We will first need to store all the Spring Boot latest Documents into RAG DB.

   - For this purpose we will use ChromaDB as Vector Store.

   - Before storing data into DB we need to first read the latest Spring Boot documentation from https://docs.spring.io/spring-boot/index.html
   URL and load the content into ChromaDB.

   - When loading we will load the page content and its source (URL of the page from which the conetent was stored)

### The Agent Component

   - Then we will create a tool which retrive the context information from this DB for the User query and use that information to make LLM call to finalize the result.

### Flow


  User --> (Query) --> Agent --> Decide Valid Spring Boot Query
                             --> No --> Return
                             --> Yes --> Retrive Context and Its source
                                     --> Make LLM Call
                                     --> Respond to User



In [ ]:
# RAG Agent Implementation Tech dependencies.

# Install Dependencise

!pip install -qU langchain
!pip install -qU langchain-openai
!pip install -qU langchain-community
!pip install -qU chromadb
!pip install -qU langchain-chroma
!pip install -qU nest-asyncio

### Tech Stack Pre-Requiste

  - Python 3.10+ < Python 4
  - LangChain Framework
  - ChromaDB
  - OpenAI API Key
  - LangSmith API Key (Optinal-For Tracing LLM calls)

### Technical Implementation

#### Loader Component

  - First we need to load Spring Boot latest documentation into ChromaDB.

  - Before loading into DB we need to read this information from https://docs.spring.io/spring-boot/index.html this official website and seprate the main content from other extra information.

  - Our loader component will use Langchain's **RecursiveUrlLoader** to crawl through the index page to deep into other pages of this website and get the content alone removing other nav, header, footer information.

  - The current code will go 3 level deep child URL from given parent URL and find all the conent and seperated those conents docuemnts along with its meta data information.

In [ ]:
# Define OpenAI Embedding model to

import os
import pprint
import nest_asyncio
from bs4 import BeautifulSoup

from google.colab import userdata

from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import RecursiveUrlLoader
#from langchain_community.document_loaders.sitemap import SitemapLoader

nest_asyncio.apply()

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
os.environ["LANGSMITH_API_KEY"] = userdata.get('LANGSMITH_API_KEY')

def extract_spring_boot_content(html: str) -> str:
    soup = BeautifulSoup(html, "html.parser")

    # Find the article with class "doc"
    article = soup.find('article', {'class': 'doc'})

    if article:
        # Remove breadcrumbs and pagination navigation
        for element in article.find_all(['nav']):
            element.decompose()

        # Also remove breadcrumbs-container div if you don't want it
        breadcrumbs = article.find('div', {'class': 'breadcrumbs-container'})
        if breadcrumbs:
            breadcrumbs.decompose()

        # Get clean text
        return article.get_text(separator='\n', strip=True)

    # Fallback if article not found
    return ""



sitemap_url = "https://docs.spring.io/spring-boot/index.html"

# loader = SitemapLoader(
#     web_path=sitemap_url,
#     filter_urls=["docs.spring.io"],
#     parsing_function=parse_spring_docs,
#     requests_per_second=2  # Be a good citizen to avoid getting blocked
# )

loader = RecursiveUrlLoader(
    url=sitemap_url,
    max_depth=3,
    extractor=extract_spring_boot_content,
    prevent_outside=True,
    # Remove link_regex to allow all links within the same domain
    # Or use a more permissive pattern that matches the domain
    #link_regex=r"https://docs\.spring\.io/spring-boot/.*",
    timeout=10
)

print("Loading Spring Boot 4 documentation pages...")
documents = loader.load()
print(f"Successfully loaded {len(documents)} pages.")

url_set = set()
for doc in documents:
    print(doc.metadata.get('source'))
    if(doc.metadata.get('source') == 'https://docs.spring.io/spring-boot/community.html'):
        print(doc.page_content)






#### Loader Component Continued ...

  - Once main content documents are seperated then we need to split them further and then store them into DB.

  - To split the documents further we will use Langchain's **RecursiveCharacterTextSplitter** with defined chunk size of 1000 and chun_overlap 0f 200

  - Once these documents are split into chunks we will create each chunks respective embeddings (Vector representation of each chinks words)  using OpenAIEmbedding model **text-embedding-3-small** and store those embedding vectors in ChromaDB.

In [ ]:
# Embed and Load documents into Chroma DB

import os
import pprint

from google.colab import userdata

from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter

embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small",
    chunk_size=1000
)

vector_store = Chroma(
    collection_name = "spring_boot_4_document_collection",
    embedding_function = embeddings,
    persist_directory = "./sample_data/spring_boot_4_document_db"
)

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # chunk size (characters)
    chunk_overlap=200,  # chunk overlap (characters)
    add_start_index=True,  # track index in original document
)


all_splits = text_splitter.split_documents(documents)

print(f"Split blog post into {len(all_splits)} sub-documents.")

# To load 5000 document at once to avoid chromaDB limitation of loading >5,461 documents at once.
batch_size=5000
# To store VectorIDs after adding documents to vector DB
vector_ids = []
for i in range(0, len(all_splits),batch_size):
    batch_docs = all_splits[i:i+batch_size]
    vec_ids = vector_store.add_documents(documents=batch_docs)
    vector_ids.extend(vec_ids)

vect_id_range = vector_ids[:4]
print(vect_id_range)

#document_ids = vector_store.add_documents(documents=all_splits)

#print(document_ids[:3])

## The Agent Component

  - Final steps is to create Agent which uses the Context information which we stored in ChromaDB and make LLM call to answer user query.

  - Our agent will have a tool which retrive context information from ChromaDB for the user query. If the query is relevant to Spring Boot 4 and its find the conext information it will return documentation contents along with source (page URL from which that conext information was retrived).

  - Here our tool might return multiple source information which match user query text as we are using vectorDB similarity_search with k=3. It will try to identify 3 nearest neighbors (documents) which match user input query text.

  - If its found the matching docuemnts from DB all of them will be sent to LLM then LLM will consolidate and provide fianl answer to the user query using this retrieved context information.

In [ ]:
from langchain.agents import create_agent
from langchain.tools import tool
from langchain.chat_models import init_chat_model

# 1. Enable tracing (REQUIRED)
os.environ["LANGCHAIN_TRACING_V2"] = "true"

# 2. Set the API Key (Correct standard name is LANGCHAIN_API_KEY)
os.environ["LANGSMITH_API_KEY"] = userdata.get('LANGSMITH_API_KEY')

# 3. (Optional) Group these runs into a specific project name
os.environ["LANGCHAIN_PROJECT"] = "Spring_Boot_Agent_Test"

@tool(response_format="content_and_artifact")
def retrive_context(query: str):
    """
    Retrieve information from Spring Boot 4 docs.
    Returns text and the specific source URL for each chunk.
    """
    # 1. Get raw results from your database
    retrieved_docs = vector_store.similarity_search(query, k=3)

    formatted_context = []
    for doc in retrieved_docs:
        source_url = doc.metadata.get('source', 'No URL available')
        snippet = f"SOURCE: {source_url}\nCONTENT: {doc.page_content}"
        formatted_context.append(snippet)

    content = "\n\n---\n\n".join(formatted_context)

    # 3. Return content for LLM and full docs as artifact
    return content, retrieved_docs

def run_agent_query(query: str):
    """
    Standardized function to handle agent execution.
    Easy to call from a CLI, a Web API, or a Colab loop.
    """
    responses = []
    for event in agent.stream(
        {"messages": [{"role": "user", "content": query}]},
        stream_mode="values"
    ):
        # In production, you might return the response rather than just printing
        responses.append(event["messages"][-1])

    return responses

# Agent Definition and declaration



tools = [retrive_context]

system_prompt=(
    """
    You are an expert on Spring Boot 4.0.1.
    Use the provided tool to find context for the user's query.

    CRITICAL INSTRUCTION: For every fact or code snippet you provide,
    you MUST cite the specific SOURCE URL provided in the context.

    Format your response like this:
    - Answer text...
    - Source: [URL here]

    If the query is not related to Spring Boot, say you don't understand.
    """
)

model = init_chat_model("gpt-4.1")
agent = create_agent(model, tools, system_prompt=system_prompt)


# user_query = (
#     """
#      Give me list of Managed Dependency Coordinates in spring boot 4.0.1
#     """
# )

# for event in agent.stream(
#     {"messages":[{"role":"user", "content": user_query}]},
#     stream_mode="values"
# ):

#     event["messages"][-1].pretty_print()

# Usage in Colab:
result = run_agent_query(input("Ask something: "))
for message in result:
    message.pretty_print()
#print(result)

# Usage in Production API:
# @app.post("/chat")
# def chat(payload: dict):
#     return run_agent_query(payload["query"])

